In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 684492718121972122
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6267797504
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5998063462702606584
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:06:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [2]:
import numpy as np
import os
import pandas as pd

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [15]:
actions = ['0','1','2','3','4','5','6','7','8','9','10','가렵다','개','공원','금요일',
    '내년','내일','냄새나다','누나','동생','목요일','물','아래','바다','배고프다','병원',
    '불','산','삼키다','선생님','수요일','아빠','아파트','앞','어제','어지럽다','언니','엄마',
    '오늘','오른쪽','오빠','올해','왼쪽','월요일','위','음식','일요일','자동차','작년','집',
    '친구','택시','토요일','학교','형','화요일','화장실',
    '가다','감사합니다','괜찮습니다','끝','나','남자','내리다','당신','돕다','맞다',
    '모르다','무엇','미안합니다','반드시','부탁합니다','빨리','수고','수화','슬프다',
    '싫다','아니다','안녕하세요','알다','없다','여자','오다','있다','잘','좋다',
    '주다','키우다','타다'
]
len(actions)

89

In [10]:
x_data1 = np.load('dataset/train_data1_cnn_feature.npy')
x_data2 = np.load('dataset/train_data2_cnn_feature.npy')

y_data1 = np.load('dataset/train_data1_cnn_label.npy')
y_data2 = np.load('dataset/train_data2_cnn_label.npy')

In [11]:
x_data1.shape, x_data2.shape, y_data1.shape, y_data2.shape

((1140, 20, 45, 80, 3), (660, 20, 45, 80, 3), (1140,), (660,))

In [12]:
x = np.concatenate([x_data1, x_data2], axis=0)
y = np.concatenate([y_data1, y_data2], axis=0)

In [13]:
x.shape, y.shape

((1800, 20, 45, 80, 3), (1800,))

In [8]:
import tensorflow as tf
from tensorflow import keras

In [16]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(y, num_classes=len(actions))

In [17]:
x.shape, y.shape

((1800, 20, 45, 80, 3), (1800, 89))

In [19]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2022)

In [78]:
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPool2D, BatchNormalization, Dropout, Flatten, LSTM, Dense

keras.backend.clear_session()

model = keras.models.Sequential()

model.add(TimeDistributed(Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='swish'),
                         input_shape=x.shape[1:]))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPool2D(pool_size=(4,4))))
model.add(TimeDistributed(Dropout(0.7)))

model.add(TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='swish')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPool2D(pool_size=(2,2))))
model.add(TimeDistributed(Dropout(0.7)))
          
model.add(TimeDistributed(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='swish')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPool2D(pool_size=(2,2))))
model.add(TimeDistributed(Dropout(0.5)))
          
model.add(TimeDistributed(Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='swish')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPool2D(pool_size=(2,2))))
model.add(TimeDistributed(Dropout(0.5)))

model.add(TimeDistributed(Flatten()))

model.add(LSTM(128, recurrent_dropout=0.3))

model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 20, 45, 80, 16)   448       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 20, 11, 20, 16)   0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 20, 11, 20, 16)   0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 20, 11, 20, 32)   4640      
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 20, 5, 10, 32)    0

In [79]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1, min_delta=0)
mc = ModelCheckpoint('model/model_cnn_ver01.h5', monitor='val_loss', verbose=1, save_best_only=True)

with tf.device("/device:GPU:0"):
    history = model.fit(x_train, y_train, validation_split=0.2, epochs=500, batch_size=4, verbose=1, callbacks=[es, mc])

Epoch 1/500
288/288 [==============================] - ETA: 0s - loss: 4.4178 - accuracy: 0.0191
Epoch 00001: val_loss improved from inf to 5.27819, saving model to model\model_cnn_ver01.h5
288/288 [==============================] - 34s 115ms/step - loss: 4.4178 - accuracy: 0.0191 - val_loss: 5.2782 - val_accuracy: 0.0035
Epoch 2/500
288/288 [==============================] - ETA: 0s - loss: 4.0527 - accuracy: 0.0243
Epoch 00002: val_loss did not improve from 5.27819
288/288 [==============================] - 33s 114ms/step - loss: 4.0527 - accuracy: 0.0243 - val_loss: 5.7772 - val_accuracy: 0.0139
Epoch 3/500
288/288 [==============================] - ETA: 0s - loss: 3.9496 - accuracy: 0.0208
Epoch 00003: val_loss did not improve from 5.27819
288/288 [==============================] - 34s 117ms/step - loss: 3.9496 - accuracy: 0.0208 - val_loss: 6.1367 - val_accuracy: 0.0278
Epoch 4/500
288/288 [==============================] - ETA: 0s - loss: 4.2039 - accuracy: 0.0208
Epoch 00004: va

288/288 [==============================] - ETA: 0s - loss: 3.7677 - accuracy: 0.0451
Epoch 00029: val_loss did not improve from 3.99914
288/288 [==============================] - 33s 116ms/step - loss: 3.7677 - accuracy: 0.0451 - val_loss: 4.2764 - val_accuracy: 0.0035
Epoch 30/500
288/288 [==============================] - ETA: 0s - loss: 3.7516 - accuracy: 0.0486
Epoch 00030: val_loss did not improve from 3.99914
288/288 [==============================] - 33s 116ms/step - loss: 3.7516 - accuracy: 0.0486 - val_loss: 4.0606 - val_accuracy: 0.0069
Epoch 31/500
288/288 [==============================] - ETA: 0s - loss: 3.7058 - accuracy: 0.0512
Epoch 00031: val_loss did not improve from 3.99914
288/288 [==============================] - 32s 111ms/step - loss: 3.7058 - accuracy: 0.0512 - val_loss: 4.6821 - val_accuracy: 0.0104
Epoch 32/500
288/288 [==============================] - ETA: 0s - loss: 3.7119 - accuracy: 0.0590
Epoch 00032: val_loss did not improve from 3.99914
288/288 [======

288/288 [==============================] - ETA: 0s - loss: 2.8530 - accuracy: 0.1927
Epoch 00057: val_loss did not improve from 3.57913
288/288 [==============================] - 29s 101ms/step - loss: 2.8530 - accuracy: 0.1927 - val_loss: 4.1242 - val_accuracy: 0.0764
Epoch 58/500
288/288 [==============================] - ETA: 0s - loss: 2.8132 - accuracy: 0.1780
Epoch 00058: val_loss did not improve from 3.57913
288/288 [==============================] - 30s 104ms/step - loss: 2.8132 - accuracy: 0.1780 - val_loss: 3.7428 - val_accuracy: 0.1007
Epoch 59/500
288/288 [==============================] - ETA: 0s - loss: 2.8077 - accuracy: 0.2066
Epoch 00059: val_loss did not improve from 3.57913
288/288 [==============================] - 28s 97ms/step - loss: 2.8077 - accuracy: 0.2066 - val_loss: 3.8302 - val_accuracy: 0.0833
Epoch 60/500
288/288 [==============================] - ETA: 0s - loss: 2.7499 - accuracy: 0.2040
Epoch 00060: val_loss improved from 3.57913 to 3.35483, saving mode